In [0]:
#import pyspark and create spark session
import pyspark
from pyspark.sql import SparkSession

#create spark session
spark = SparkSession.builder.getOrCreate()
from pyspark.sql import functions as F

In [0]:
#dbutils.fs.mv('/FileStore/tables/skill2vec_50K_csv.gz', '/FileStore/tables/skill2vec_50K.csv.gz')

In [0]:
#read the input file into a Dataframe
jd = spark.read.csv('/FileStore/tables/skill2vec_50K.csv.gz')

#Sanity check - Verify if the data is loaded correctly
print("number of records:" ,jd.count())

number of records: 50000


In [0]:
#create a dataframe with column 0 - identifier and column 1 - the skills list
jd_columns = jd.columns[1:]
jdlistDF = jd.select(jd.columns[0], F.array( jd_columns ).alias( 'jd_skills_list') )

#rename the first column as jd_identifier
jdlistDF = jdlistDF.withColumnRenamed('_c0','jd_identifier')

#remove all the nulls and create and new column with only data
jd_non_nullDF = jdlistDF.withColumn ( 'jd_skills_list_not_null' , F.array_except('jd_skills_list' , F.array(F.lit(None))))

#count the number of skills listed in the job description and add a column with the count of skills
jd_non_nullDF_count = jd_non_nullDF.withColumn ( 'Num_skills' , F.size( 'jd_skills_list_not_null' ) )

In [0]:
#Q1: sanity check: to see if there are any null values in the jd_identifier
print("Number of rows in job identifier is with null:", jd_non_nullDF.where(jd_non_nullDF.jd_identifier == 'null').count())

Number of rows in job identifier is with null: 0


In [0]:
#Question 1:
#Programmatically confirm that the number of job descriptions is as expected (i.e. that
#there are 50,000 distinct job numbers / identifers / description in the skill2vec 50K dataset).
print("Distinct number of job descriptions: ", jd_non_nullDF.select('jd_identifier').distinct().count())

Distinct number of job descriptions:  50000


In [0]:
#Q2 - Sanity check: Every JD identifier has at least one skill
print("Number of jobs without any skills :", jd_non_nullDF_count.where(jd_non_nullDF_count.Num_skills < 1).count() )

Number of jobs without any skills : 0


In [0]:
#Question 2:
#Work out the frequencies with which distinct skills are mentioned in job descriptions,
#and present the top 10 (in order of decreasing frequency) skills in your report, alongside
#the frequency of each across the entire dataset. I.e. if your dataset consists of the
#following:

#Explode the array into individual records
ind_jd_DF = jd_non_nullDF_count.withColumn("Skills", F.explode(F.col('jd_skills_list_not_null')))

#Groupby to find the count of individual job skills
ind_jd_count_DF = ind_jd_DF.groupBy('Skills').count()

#rename column name from count to Freq
ind_jd_count_DF = ind_jd_count_DF.withColumnRenamed('count','Freq') 

#Display the top 10 skills with highest frequency
ind_jd_count_DF.sort('Freq', ascending=False).show(10,truncate = False)

+--------------------+----+
|Skills              |Freq|
+--------------------+----+
|Java                |1911|
|Javascript          |1770|
|Sales               |1705|
|Business Development|1545|
|Web Technologies    |1313|
|Communication Skills|1305|
|development         |1238|
|Marketing           |1184|
|Finance             |1078|
|HTML                |1067|
+--------------------+----+
only showing top 10 rows



In [0]:
#Question 3:
#Find the 5 most frequent numbers of skills in JDs across the dataset. I.e. given the
#example with JD1, JD2 and JD3 above, the expected result would be:

#Group by on number of skills to find the count of jobs that have the same number of skills
num_skills_count = jd_non_nullDF_count.groupBy('Num_skills').count()

#Show the top 5 most frequent number of skills listed in JD
num_skills_count.sort('count', ascending=False).show(5,truncate = False)

+----------+-----+
|Num_skills|count|
+----------+-----+
|10        |10477|
|5         |3432 |
|6         |3405 |
|1         |3386 |
|7         |3345 |
+----------+-----+
only showing top 5 rows



In [0]:
#Question 4:
#So far, you’ve explored the dataset in its original form. Check how the distribution of
#the frequencies with which distinct skills are mentioned in JDs changes if you lower case
#all the skills. As in question 2, present the top 10 (in order of decreasing frequency)
#skills in your report.

#Create a new column with individual columns in lower case
ind_jd_lower_DF = ind_jd_DF.withColumn("Skills_lower", F.lower(F.col('Skills')))

#groupby to find the count of the jobs in lower case
ind_jd_lower_count_DF = ind_jd_lower_DF.groupBy('Skills_lower').count()

#Display the top 10 skills
ind_jd_lower_count_DF.sort('count', ascending=False).show(10,truncate = False)

+--------------------+-----+
|Skills_lower        |count|
+--------------------+-----+
|java                |2759 |
|javascript          |2738 |
|sales               |2680 |
|business development|2108 |
|marketing           |1809 |
|sql                 |1564 |
|jquery              |1547 |
|html                |1539 |
|communication skills|1537 |
|bpo                 |1530 |
+--------------------+-----+
only showing top 10 rows



In [0]:
#dbutils.fs.mv('/FileStore/tables/TechnologySkills.txt', '/FileStore/tables/Technology_Skills.txt')

In [0]:
#Question 5:
#To gain some additional information about the sought after skills, you’d like to join
#the (lower cased) skills from JDs with the skills listed in the Example column in the
#O*NET dataset (don’t forget to lower case the example column!). Find the change in
#the number of skills before and after the join (i.e. report the number of original skills
#and the skills that are both in the JD dataset and the O*NET dataset { reporting two
#separate numbers).

#read the input file into a Dataframe
Tech_skills = spark.read.format('csv') \
   .option('header', 'true') \
   .option('delimiter', '\t') \
   .load('dbfs:/FileStore/tables/Technology_Skills.txt')

#Convert Example column to lower case
Tech_skills_lower = Tech_skills.withColumn("Example_lower", F.lower(F.col('Example')))

#To print the count before join
print("Count of JD skills before join:" , ind_jd_lower_DF.select('Skills_lower').count() )

#Join the 2 dataframe using skills columns
join_jd = ind_jd_lower_DF.join(Tech_skills_lower , ind_jd_lower_DF['Skills_lower'] == Tech_skills_lower['Example_lower'] , 'inner')

#To print the count after join
print("Count of JD skills after join with Tech skills = ", join_jd.count())

Count of JD skills before join: 463803
Count of JD skills after join with Tech skills =  1101498


In [0]:
#Question 6:
#The join you performed in Question 5 gives you access to the \Commodity Title" column. 
#Find the 10 most frequent \Commodity Title"s across all the job descriptions.
#I.e. using the example from Question 2, the output should be:

#Group by on Commodity Title
join_jd_count = join_jd.groupby('Commodity Title').count()

#Sort on descending order and display only the top 10
join_jd_count.sort('count', ascending=False).show(10,truncate = False)

+-------------------------------------------------+------+
|Commodity Title                                  |count |
+-------------------------------------------------+------+
|Object or component oriented development software|324521|
|Web platform development software                |298754|
|Operating system software                        |190926|
|Development environment software                 |53013 |
|Data base management system software             |44132 |
|Analytical or scientific software                |33552 |
|Web page creation and editing software           |31682 |
|Data base user interface and query software      |29436 |
|Spreadsheet software                             |18568 |
|File versioning software                         |13846 |
+-------------------------------------------------+------+
only showing top 10 rows

